## Carrega bibliotecas necessárias e a base de dados processada

In [105]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("../data/dataset_categorizado.csv")

## Qual a distribuição entre os sensíveis às camapanhas e os não sensíveis?

In [3]:
# Avalia a distribuição de grupos
for res in [0, 1]:
    subset = dataset.loc[dataset["Response"] == res]
    grupos = set(subset["grupo_cliente"].values)
    
    print(f"Clientes com {res} para campanha")
    for grupo in grupos:
        qntd = list(subset["grupo_cliente"].values).count(grupo)
        pp = round(qntd * 100 / len(subset), 2)
        print(f"Grupo {grupo} ({qntd}): {pp}%")
    print("")

Clientes com 0 para campanha
Grupo 0 (242): 25.47%
Grupo 1 (197): 20.74%
Grupo 2 (142): 14.95%
Grupo 3 (170): 17.89%
Grupo 4 (199): 20.95%

Clientes com 1 para campanha
Grupo 0 (23): 19.17%
Grupo 1 (26): 21.67%
Grupo 2 (13): 10.83%
Grupo 3 (27): 22.5%
Grupo 4 (31): 25.83%



In [114]:
# Avalia a distribuição para tipos de produtos
campos = {
    "MntWines": 0,
    "MntFruits": 0,
    "MntMeatProducts": 0,
    "MntFishProducts": 0,
    "MntSweetProducts": 0,
    "MntGoldProds": 0
}

# Avalia a distribuição em cada grupo
for grupo in set(dataset["grupo_cliente"].values):
    for res in [0, 1]:
        
        print(f"(GRUPO {grupo}): Clientes com {res} para campanha")
        subset = dataset.loc[
            (dataset["Response"] == res)
            & (dataset["grupo_cliente"] == grupo)
        ]
        print(f"Total de registros: {len(subset)}")
    
        # Computa as quantidades em cada tipo de produto
        total_compras = 0
        for campo in campos.keys():
            total = sum(subset[campo].values)
            campos[campo] = total
            total_compras = total_compras + total

        # Compila as porcentagens
        for campo in campos.keys():
            pp = round(campos[campo] * 100 / total_compras, 2)
            print(f"{campo} ({campos[campo]}): {pp}%")
        print("")
    print("----------------------------\n")

(GRUPO 0): Clientes com 0 para campanha
Total de registros: 242
MntWines (61285): 64.07%
MntFruits (2501): 2.61%
MntMeatProducts (17465): 18.26%
MntFishProducts (3791): 3.96%
MntSweetProducts (2538): 2.65%
MntGoldProds (8075): 8.44%

(GRUPO 0): Clientes com 1 para campanha
Total de registros: 23
MntWines (8673): 69.36%
MntFruits (223): 1.78%
MntMeatProducts (2174): 17.39%
MntFishProducts (312): 2.5%
MntSweetProducts (282): 2.26%
MntGoldProds (841): 6.73%

----------------------------

(GRUPO 1): Clientes com 0 para campanha
Total de registros: 197
MntWines (3795): 29.9%
MntFruits (911): 7.18%
MntMeatProducts (3662): 28.86%
MntFishProducts (1288): 10.15%
MntSweetProducts (805): 6.34%
MntGoldProds (2230): 17.57%

(GRUPO 1): Clientes com 1 para campanha
Total de registros: 26
MntWines (395): 17.95%
MntFruits (155): 7.05%
MntMeatProducts (583): 26.5%
MntFishProducts (215): 9.77%
MntSweetProducts (153): 6.95%
MntGoldProds (699): 31.77%

----------------------------

(GRUPO 2): Clientes com 

In [5]:
# Avalia a distribuição as formas de compra
campos = {
    "NumDealsPurchases": 0,
    "NumWebPurchases": 0,
    "NumCatalogPurchases": 0,
    "NumStorePurchases": 0
}

for res in [0, 1]:
    print(f"Clientes com {res} para campanha")
    subset = dataset.loc[dataset["Response"] == res]
    
    # Computa as quantidades em cada forma de compra
    total_formas = 0
    for campo in campos.keys():
        total = sum(subset[campo].values)
        campos[campo] = total
        total_formas = total_formas + total
        
    # Compila as porcentagens
    for campo in campos.keys():
        pp = round(campos[campo] * 100 / total_formas, 2)
        print(f"{campo} ({campos[campo]}): {pp}%")
    print("")

Clientes com 0 para campanha
NumDealsPurchases (2104): 18.38%
NumWebPurchases (3181): 27.79%
NumCatalogPurchases (1430): 12.49%
NumStorePurchases (4732): 41.34%

Clientes com 1 para campanha
NumDealsPurchases (319): 19.0%
NumWebPurchases (538): 32.04%
NumCatalogPurchases (270): 16.08%
NumStorePurchases (552): 32.88%



## Encontra um modelo classificativo

In [103]:
# Prepara os dados para modelagem
y, x = [], []

# Define a regra para equalizar a frequência das classes
# Utiliza o ID do registro para coletar únicos
dist = {"1": 120, "0": 120}
ids = []
total = sum([dist[k] for k in dist.keys()])

while len(x) < total:
    i = np.random.randint(0, len(dataset))
    if dataset["ID"].values[i] in ids:
        continue
        
    classe = dataset["Response"].values[i]
    if dist[str(classe)] > 0:
        # Salva nas listas para modelagem
        y.append(classe)
        x.append([
            dataset["encoder_escolaridade"].values[i],
            dataset["encoder_estado_civil"].values[i],
            dataset["encoder_nascimento"].values[i],
            #dataset["Kidhome"].values[i],
            #dataset["Teenhome"].values[i],
            dataset["encoder_renda"].values[i]
        ])
        
        # Atualiza a distruibuição e a lista de ids
        dist[str(classe)] = dist[str(classe)] - 1
        ids.append(dataset["ID"].values[i])

# Separa o conjunto de treino e de teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=0)

# Executa a classificação
classificador = GaussianNB()
classificador.fit(x_treino, y_treino)

# Avalia acurácia
y_pred = classificador.predict(x_teste)
acuracia = accuracy_score(y_teste, y_pred)

# Para a matriz de confusão, o melhor é ter poucos erros com falso positivo
mc = confusion_matrix(y_teste, y_pred)
fp = mc[0][1] / len(y_teste)

print(f"Encontrada acurácia de {round(acuracia, 2)}")
print(f"Total de falsos positivos de {round(fp, 2)}")

Encontrada acurácia de 0.71
Total de falsos positivos de 0.08


### Persistindo o melhor teste

In [111]:
# Salva o modelo de classificação
file = open("../models/classificador.pickle", "wb")
pickle.dump(classificador, file)
file.close()

# Salva um txt com informações
file = open("../models/classificador.txt", "w+")
file.writelines([
    f"Modelo de classificação salvo em {datetime.now().isoformat()}.\n",
    f"Encontrada acurácia de {round(acuracia, 2)}\n",
    f"Total de falsos positivos de {round(fp, 2)}\n",
    "Campos utilizados:\n",
    "> encoder_escolaridade\n",
    "> encoder_estado_civil\n",
    "> encoder_nascimento\n",
    "> encoder_renda\n"
])
file.close()